# Reproduce Gary 1975 paper result of Alfven instabilities

In [1]:
import numpy as np
import scipy.optimize
from py_vlasov.util import zp, kzkp, list_to_complex, real_imag
from py_vlasov.wrapper import oblique_wrapper, disp_det, input_gen
from py_vlasov.transport_ratios import transport_ratios
from py_vlasov.follow_parameter import follow_k
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3.5
mpl.rcParams['ytick.minor.size'] = 3.5
mpl.rcParams['xtick.major.width'] = 1.1
mpl.rcParams['ytick.major.width'] = 1.1
mpl.rcParams['xtick.minor.width'] = .8
mpl.rcParams['ytick.minor.width'] = .8
mpl.rcParams['legend.fontsize'] = 14
#xtick.major.size     : 3.5      # major tick size in points
#xtick.minor.size     : 2      # minor tick size in points
#xtick.major.width    : 0.8    # major tick width in points
#xtick.minor.width    : 0.6    # minor tick width in points
%matplotlib inline

In [4]:
def generate_k_arr(start_k, target_k, log_incrmt = None, lin_incrmt = None):
    assert log_incrmt or lin_incrmt
    if log_incrmt:
        n = int(np.log(target_k/start_k) / np.log(1 + log_incrmt))
        ret = np.arange(n + 1) * np.log(1 + log_incrmt)
        return np.exp(ret)  * start_k
    else:
        n = (target_k - start_k) / lin_incrmt
        if n - int(n) > 1 - 1e-5:
            n = int(n) + 1
        return start_k + np.arange(n + 1) * lin_incrmt
    
def fast_freq(k, theta, betap):
    kz, kp = kzkp(k, theta)
    cos_theta = kz / k
    kdi = k / np.sqrt(betap)
    return kdi * np.sqrt(1 + kdi**2 * cos_theta**2)

def whistler_freq(k, theta, betap):
    kz, kp = kzkp(k, theta)
    cos_theta = kz / k
    kdi = k / np.sqrt(betap)
    return 1836 * kdi**2 / (1836 + kdi**2) * cos_theta   

def alfven_freq(k, theta, betap):
    kz, kp = kzkp(k, theta)
    return kz / np.sqrt(betap)

def kdi_2_krhoi(kdi, betap):
    return kdi * np.sqrt(betap)

In [5]:
aol = np.sqrt(2.65e-8)
print('aol = ', aol)
tc_tp = 2
th_tc = 6
nc_np = 0.95
nh_np = 1 - nc_np
vc_va = 0#2.42
vh_va = -vc_va * nc_np / nh_np
print('vh_va = {0:.3g}'.format(vh_va))

betap = 0.25
theta = 80


t_list=[1., tc_tp, th_tc * tc_tp]
a_list=[1., 1., 1.]
n_list=[1., nc_np, nh_np] 
q_list=[1.,-1., -1.]
m_list=[1., 1./1836., 1./1836]
v_list=[0., vc_va, vh_va]
n = 6
method = 'pade'

k = 0.2
kz, kp = kzkp(k, theta)

param = [kz, kp, betap, t_list, a_list, n_list, q_list,
         m_list, v_list, n, method, aol]

aol =  0.000162788205961
vh_va = 0


In [7]:
# seed_freq = kz / np.sqrt(betap)
seed_freq = fast_freq(k, theta, betap)
f = lambda wrel:\
    real_imag(oblique_wrapper(list_to_complex(wrel), kz, kp, betap,
                              t_list, a_list, n_list, q_list, \
                              m_list, v_list, method = method, n = n, \
                              aol=aol))
guess = seed_freq 
print('seed_freq = {0:.3g}'.format(guess))

wr, wi = scipy.optimize.fsolve(f, real_imag(guess))
print(wr)
print(wi)

seed_freq = 0.401
0.0707387571061
-0.000252066958991
